<a href="https://colab.research.google.com/github/achiral/mosaic_art/blob/main/gh_mosaic_art.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### preparation

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# make dataset directories
!mkdir ./dataset
!mkdir /content/output

In [ ]:
# material: blocks.zip
!cp "./gdrive/MyDrive/dataset/blocks.zip" "./dataset"

# target image (*.png, *.jpg)
!cp "./gdrive/MyDrive/dataset/target.png" "./dataset"

In [ ]:
# unzip material dataset
!unzip ./dataset/blocks.zip -d ./dataset

# count data
!find dataset/* -type f | wc -l

### run mosaic art

In [ ]:
%cd /content

In [ ]:
import numpy as np
import cv2
import glob

In [ ]:
# IMG_PATH = './images/*.png'          # material image path
IMG_PATH = R"./dataset/blocks/*.png"  # material image path

INPUT_PATH = R"./dataset/target.png"  # target image path
MULTIPLY = 2                           # magnify target image

In [ ]:
# files = glob.glob('./images/*.png')
files = glob.glob(IMG_PATH)
size = 50    # resize into 50x50
 
small_img = []
 
for file in files:
    image = cv2.imread(file)
    image = cv2.resize(image, (size, size))
    image = image /255
    small_img.append(image)
 
small_img = np.asarray(small_img)
pic_filename = INPUT_PATH                 # target image path
 
big_img = cv2.imread(pic_filename)
height = (big_img.shape[0] - big_img.shape[0] % size) * 2 * MULTIPLY
width = (big_img.shape[1] - big_img.shape[1] % size) * 2 * MULTIPLY
 
big_img = big_img / 255
big_img = np.asarray(big_img)
big_img = cv2.resize(big_img, (width, height))
 
for i in range(int(height / size)):
    print('%d / %d' %(i + 1, int(height / size)))
 
    for j in range(int(width / size)):
        cut = big_img[size * i: size * (i + 1), size * j: size * (j + 1)]
        losses = [] 
        
        # search marial image (small_img) similar to target image color
        for s_img in small_img:
            loss = cut - s_img
            loss = loss * loss
            loss = np.sum(loss)
            losses.append(loss) 
     
        losses = np.asarray(losses)
        big_img[size * i: size * (i + 1), size * j: size * (j + 1)] = small_img[np.argmin(losses)]
 
big_img = (big_img * 255).astype(np.int64)
cv2.imwrite('out.png', big_img)

### colab2drive

In [ ]:
!cp "out.png" "./gdrive/MyDrive/dataset/out.png"
print("----- finished -----")